In [1]:
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
from matplotlib.pyplot import figure
import pandas as pd

In [26]:
### INSERT VARIABLES HERE ###
assetLife = 20
gearboxLife = 10
tstrike = 10
numberOfTurbines = 100 #number of turbines
inflationRate = 1.05 #Assuming 5% inflation rate

elecPriceSigma = 0.18
gearboxCost = 1 #£MM per turbine
gearboxSigma = 0.15

totalSigma = 0.085

#math.sqrt((elecPriceSigma**2)+(gearboxSigma**2))

ratedCap = 8 #MW
costPerMW = 0.9 #£1.1mil per MW
capFactor = 0.3 #Output is 30% of RC
availability = 0.95 #Assume turbines are on 95% of the time

initElecPrice = 55 #£/MWh

riskAdjRate = 0.07 #risk-adjusted discount rate
riskFreeRate = 0.03

In [29]:
### FUNCTIONS FOR BAU CASE ###

def BAUfindSetupCost(ratedCap, numberOfTurbines, costPerMW):
    setupCost = numberOfTurbines * costPerMW * ratedCap
    return setupCost

def BAUfindNetYield(numberOfTurbines, ratedCap, capFactor, availability):
    grossYield = numberOfTurbines * ratedCap * capFactor * 8760 #8760 hours in a year
    netYield = availability * grossYield
    return netYield

def BAUfindCashIn(assetLife, gearboxLife, initElecPrice, inflationRate, netYield, tstrike):
    
    elecPrice = np.zeros(gearboxLife + tstrike) #initialise arrays
    revenue = np.zeros(gearboxLife + tstrike)
    opCosts = np.zeros(gearboxLife + tstrike)
    profit = np.zeros(gearboxLife + tstrike)
    elecPrice[0] = initElecPrice
    
    for i in range (0,assetLife):
        if(i > 0):
            elecPrice[i] = elecPrice[i-1]*inflationRate
            
        revenue[i] = netYield * elecPrice[i]
        opCosts[i] = 0.3 * revenue[i] #Assuming operation costs are 30% of revenue
        profit[i] = (revenue[i] - opCosts[i]) * pow(10, -6) #profit in £MM
            
    
    return profit

def BAUfindPV(r, setupCost, assetLife, profit):
    PV = np.zeros(len(profit)) #initialise PV array
    PV[0] = -setupCost #initial cash outflow due to setup
    for i in range (1,tstrike):
        PV[i] = profit[i] / pow((1 + r), i)
        
    print("BAU Present Values:", np.around(PV, 2), '\n')

    return PV

In [30]:
### CALCULATIONS FOR BAU CASE ###

### Calculate replacement cost ###
BAUsetupCost = BAUfindSetupCost(ratedCap, numberOfTurbines, costPerMW)

### Calculate net yield ###
BAUnetYield = BAUfindNetYield(numberOfTurbines, ratedCap, capFactor, availability)

### Calculate cash in-flows ###
BAUprofit = BAUfindCashIn(assetLife, gearboxLife, initElecPrice, inflationRate, BAUnetYield, tstrike)

### Calculate Present values ###
BAUPV = BAUfindPV(riskAdjRate, BAUsetupCost, assetLife, BAUprofit)

### Calculate NPV ###
BAUNPV = np.sum(BAUPV)
print("BAU Net Present Value = £%.2f million" % BAUNPV)

BAU Present Values: [-720.     75.46   74.05   72.66   71.31   69.97   68.66   67.38   66.12
   64.89    0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.  ] 

BAU Net Present Value = £-89.50 million


In [18]:
def findReplacementCost(numberOfTurbines, gearboxCost):
    replacementCost = gearboxCost * numberOfTurbines
    return replacementCost

In [19]:
def findNetYield(numberOfTurbines, ratedCap, capFactor, availability):
    grossYield = numberOfTurbines * ratedCap * capFactor * 8760 #8760 hours in a year
    netYield = availability * grossYield
    return netYield

In [20]:
def findCashIn(assetLife, gearboxLife, initElecPrice, inflationRate, netYield):
    
    elecPrice = np.zeros(gearboxLife) #initialise arrays
    revenue = np.zeros(gearboxLife)
    opCosts = np.zeros(gearboxLife)
    profit = np.zeros(gearboxLife)
    elecPrice[0] = initElecPrice*pow(inflationRate,tstrike) # should move with inf rate at strike
    
    for i in range (0, gearboxLife):
        if(i>0):
            elecPrice[i] = elecPrice[i-1]*inflationRate
        
        revenue[i] = netYield * elecPrice[i]
        opCosts[i] = 0.3 * revenue[i] #Assuming operation costs are 30% of revenue
        profit[i] = (revenue[i] - opCosts[i]) * pow(10, -6) #profit in £MM
        
        #if(i >= gearboxLife - 3):
            #profit[i] = (revenue[i] - opCosts[i]) * pow(0.8, i - (gearboxLife - 4)) * pow(10, -6) #profit in £MM
            #otherwise devides by decimals due to negative power ,  which grows values
            
           
    return profit

In [21]:
def findPV(riskFreeRate, riskAdjRate, replacementCost, assetLife, gearboxLife, profit, tstrike):
    PV = np.zeros(gearboxLife) #initialise PV array
    
    #PV[0] = -replacementCost
    for i in range (1, assetLife - tstrike):
        PV[i] = profit[i] / pow((1 + riskFreeRate), tstrike+i) # should start from tstrike not tstart

    for i in range (assetLife - tstrike, gearboxLife):
        PV[i] = profit[i] / pow((1 + riskAdjRate), tstrike+i)

    print("Replacement case Present Values:", np.around(PV, 2), '\n')
    return PV

In [22]:
### CALCULATIONS FOR OPTION CASES ###

### Calculate replacement cost ###
replacementCost = findReplacementCost(numberOfTurbines, gearboxCost)

### Calculate net yield ###
netYield = findNetYield(numberOfTurbines, ratedCap, capFactor, availability)

### Calculate cash in-flows ###
profitOption = findCashIn(assetLife, gearboxLife, initElecPrice, inflationRate, netYield)
#print(profitOption)

### Calculate Present Values ###
PVOption = findPV(riskFreeRate, riskAdjRate, replacementCost, assetLife, gearboxLife, profitOption, tstrike)

### Calculate NPVs ###
NPVOption = np.around(np.sum(PVOption), 2)
print("Net Present Value of replacement case = £%.2f million\n" % NPVOption)

Replacement case Present Values: [  0.    95.01  96.86  98.74 100.65 102.61 104.6  106.63 108.7  110.81] 

Net Present Value of replacement case = £924.61 million



In [23]:
### BINOMIAL TREE


S = NPVOption
K = replacementCost / pow(inflationRate, tstrike)#Equivalent of exercise price in B-S
sigma = totalSigma
N = 30
dt = tstrike/N
u = np.exp(sigma*np.sqrt(dt))
d = 1/u
r = riskFreeRate
p = (np.exp(r*dt) - d)/(u-d)
q = 1-p

print(K)


stock_prices = np.zeros((N+1, N+1))
option_values = np.zeros((N+1, N+1))

for i in range(N+1):
    for j in range(i+1):
        stock_prices[j][i] = S*(u**(i-j))*(d**j)

df = pd.DataFrame(stock_prices)
df

61.39132535407592


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,924.61,971.116849,1019.962940,1071.265936,1125.149414,1181.743172,1241.183531,1303.613675,1369.183985,1438.052408,...,2591.370739,2721.713789,2858.612949,3002.397983,3153.415244,3312.028504,3478.619834,3653.590522,3837.362039,4030.377058
1,0.00,880.330367,924.610000,971.116849,1019.962940,1071.265936,1125.149414,1181.743172,1241.183531,1303.613675,...,2349.112114,2467.269826,2591.370739,2721.713789,2858.612949,3002.397983,3153.415244,3312.028504,3478.619834,3653.590522
2,0.00,0.000000,838.171289,880.330367,924.610000,971.116849,1019.962940,1071.265936,1125.149414,1181.743172,...,2129.501441,2236.612982,2349.112114,2467.269826,2591.370739,2721.713789,2858.612949,3002.397983,3153.415244,3312.028504
3,0.00,0.000000,0.000000,798.031211,838.171289,880.330367,924.610000,971.116849,1019.962940,1071.265936,...,1930.421439,2027.519478,2129.501441,2236.612982,2349.112114,2467.269826,2591.370739,2721.713789,2858.612949,3002.397983
4,0.00,0.000000,0.000000,0.000000,759.813445,798.031211,838.171289,880.330367,924.610000,971.116849,...,1749.952764,1837.973431,1930.421439,2027.519478,2129.501441,2236.612982,2349.112114,2467.269826,2591.370739,2721.713789
5,0.00,0.000000,0.000000,0.000000,0.000000,723.425929,759.813445,798.031211,838.171289,880.330367,...,1586.355504,1666.147412,1749.952764,1837.973431,1930.421439,2027.519478,2129.501441,2236.612982,2349.112114,2467.269826
6,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,688.781015,723.425929,759.813445,798.031211,...,1438.052408,1510.384836,1586.355504,1666.147412,1749.952764,1837.973431,1930.421439,2027.519478,2129.501441,2236.612982
7,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,655.795247,688.781015,723.425929,...,1303.613675,1369.183985,1438.052408,1510.384836,1586.355504,1666.147412,1749.952764,1837.973431,1930.421439,2027.519478
8,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,624.389170,655.795247,...,1181.743172,1241.183531,1303.613675,1369.183985,1438.052408,1510.384836,1586.355504,1666.147412,1749.952764,1837.973431
9,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,594.487132,...,1071.265936,1125.149414,1181.743172,1241.183531,1303.613675,1369.183985,1438.052408,1510.384836,1586.355504,1666.147412


In [24]:
for i in range(N+1):
    option_values[i][N] = max(0, stock_prices[i][N] - K)


for i in range(N-1, -1, -1):
    for j in range(i+1):
        option_values[j][i] = (p*option_values[j][i+1] + (1-p)*option_values[j+1][i+1])*np.exp(-r*dt)
        
df2 = pd.DataFrame(option_values)
df2

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,879.130188,925.179957,973.564375,1024.401057,1077.813536,1133.931559,1192.891405,1254.836204,1319.916292,1388.289567,...,2535.263292,2665.042453,2801.372056,2944.581810,3095.018009,3253.044367,3419.042897,3593.414826,3776.581568,3968.985733
1,0.000000,834.393475,878.211434,924.251970,972.627062,1023.454323,1076.857287,1132.965701,1191.915839,1253.850834,...,2293.004667,2410.598490,2534.129846,2663.897616,2800.215714,2943.413846,3093.838307,3251.852809,3417.839363,3592.199196
2,0.000000,0.000000,791.772723,833.465488,877.274121,923.305237,971.670814,1022.488465,1075.881722,1131.980330,...,2073.393994,2179.941646,2291.871221,2409.453653,2532.973504,2662.729652,2799.036011,2942.222288,3092.634773,3250.637179
3,0.000000,0.000000,0.000000,751.166332,790.835410,832.518755,876.317873,922.339378,970.695248,1021.503095,...,1874.313992,1970.848143,2072.260548,2178.796809,2290.714878,2408.285689,2531.793801,2661.538093,2797.832477,2941.006658
4,0.000000,0.000000,0.000000,0.000000,712.477566,750.219599,789.879162,831.552896,875.342307,921.354008,...,1693.845317,1781.302095,1873.180547,1969.703306,2071.104206,2177.628845,2289.535176,2407.094130,2530.590267,2660.322464
5,0.000000,0.000000,0.000000,0.000000,0.000000,675.614317,711.521318,749.253740,788.903596,830.567526,...,1530.248058,1609.476076,1692.711871,1780.157258,1872.024204,1968.535341,2069.924503,2176.437286,2288.331642,2405.878500
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,640.488888,674.648458,710.545752,748.268370,...,1381.944962,1453.713500,1529.114612,1608.331239,1691.555529,1778.989294,1870.844501,1967.343783,2068.720969,2175.221656
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,607.017776,639.513322,673.663088,...,1247.506228,1312.512649,1380.811516,1452.568663,1527.958269,1607.163275,1690.375826,1777.797735,1869.640967,1966.128153
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,575.121477,606.032406,...,1125.635725,1184.512196,1246.372783,1311.367812,1379.655173,1451.400699,1526.778567,1605.971717,1689.172292,1776.582105
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,544.724291,...,1015.158489,1068.478078,1124.502279,1183.367359,1245.216440,1310.199848,1378.475471,1450.209141,1525.575033,1604.756087


In [25]:
Co = option_values[0][0]

#Add NPV of the BAU case with the option value to find total value of the project
totalValue = Co - BAUsetupCost

print("Present Value of NPV of option case = £%.2f million\n" % S)
print("Total volitility = %.2f\n" % totalSigma)
print("Option value = £%.2f million\n" % Co)
print("Total value of project = £%.2f million\n" % totalValue)

Present Value of NPV of option case = £924.61 million

Total volitility = 0.09

Option value = £879.13 million

Total value of project = £159.13 million

